# Домашнее задание 5

Работа с базой `book_simple.db`. 

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=160L6VoP2kz8mJfpFuLWF6_SdoVcvkT-J' width="400" height="400" />
<figcaption>Структура базы book </figcaption></center>
</figure>

Ознакомиться со структурой данных можно с ипользованием, например, программы DBeaмer. 

Для работы с базой допускается использовать только возможности SQL. 
Библиотеку Pandas допускается использовать только просмотра результата.

Шаг 1.  Изучите материалы лекционных и практических занятий по теме 6.1-6.4.

Шаг 2. Получите результирующие множества из БД в соответствии с следующими условиями выборки, сортировки, группировки данных:

1. Выведите авторов, имеющихся в базе?
2. Найдите всех авторов с инициалами "С. А.". Автор в базе определяется так: И. О. Фамилия
3. Найдите все издательства, название которых состоит из двух и более слов.
4. Выведите список книг, выпущенных в издательстве "Мир".
5. Выведите количество книг, принадлежащих каждой из категорий. Результат отсортировать по возрастанию.
6. Выведите количество книг, выпущенных в каждом издательстве. В итоговом списке оставьте только те издательства, которые выпустили 3 и более книг. Результат отсортировать по убыванию.
7. Выведите список книг, автором которых являлся <<С. А. Айвазян>>.
8. В каких издательствах и сколько выпускались книги за авторством <<С. А. Айвазян>>. Результат отсортировать по убыванию.

Шаг 3. Результат каждого запроса запишите в виде таблицы в БД book.db.

Шаг 4. Опубликуйте итоговый файл БД расширения sql, прикрепив его в этом задании.

In [1]:
# Подключение библиотек
import sqlite3
import pandas as pd

In [2]:
# Подключение к БД
con = sqlite3.connect('book_simple.db')
cur = con.cursor() 

### 1. Вывод всех авторов

In [3]:
query_all_author = """
SELECT DISTINCT name
  FROM author;
"""

In [4]:
pd.read_sql(query_all_author, con)

,name
0,М. А. Айзерман
1,Ф. Т. Алескеров
2,Г. В. Савицкая
3,В. Кельтон
4,А. Лоу
...,...
1005,В. Н. Касьянов
1006,Ф. Коуэлл
1007,В. В. Ступкин
1008,М. А. Тынкевич


### 2. Вывод авторов с инициалами "С. А."

In [5]:
query_like_author = """
SELECT DISTINCT name
  FROM author
 WHERE name LIKE 'С. А.%';
"""

In [6]:
pd.read_sql(query_like_author, con)

,name
0,С. А. Айвазян
1,С. А. Орловского
2,С. А. Смоляк
3,С. А. Айвазян
4,С. А. Калмыков
5,С. А. Дубровский
6,С. А. Аристов
7,С. А. Иванков
8,С. А. Айвазяна
9,С. А. Горбатков


### 3. Издательства из двух и более слов

In [7]:
query_publisher_name = """
SELECT name
  FROM publisher
 WHERE name LIKE '% %';
"""

In [8]:
pd.read_sql(query_publisher_name, con)

,name
0,Новое знание
1,Экономическая школа
2,Высш. шк.
3,Радио и связь
4,Гyманитарный Центр
...,...
58,ИПП Куна
59,ДМК Пресс
60,Новое звание
61,ИПУ РАН


### 4. Вывод списка книг, выпущенных в издательстве "Мир"

#####  Способ 1. С применением подзапроса с JOIN . 

In [9]:
query_book_publisher_mir = """
SELECT *
  FROM (
           SELECT title,
                  name
             FROM book
                  LEFT JOIN
                  publisher ON book.publisher_id = publisher.id
       )
 WHERE name = "Мир";

"""

In [10]:
pd.read_sql(query_book_publisher_mir, con)

,title,name
0,Имитационное моделирование систем: искусство и...,Мир
1,Введение в имитационное моделирование и язык С...,Мир
2,Основы исследования операций,Мир
3,Выпуклые структуры и математическая экономика,Мир
4,Нелинейное программирование. Теория и алгоритмы,Мир
...,...,...
79,Линейная алгебра и ее применение,Мир
80,"Теория синтаксического анализа, перевода и ком...",Мир
81,"Теория синтаксического анализа, перевода и ком...",Мир
82,Введение в программирование на языке Паскаль,Мир


##### Способ 2. С применением подзапроса. 

In [11]:
query_book_publisher_mir2 = """
SELECT title
  FROM book
 WHERE publisher_id IN (
           SELECT id
             FROM publisher
            WHERE name = "Мир"
       );

"""

In [12]:
pd.read_sql(query_book_publisher_mir2, con)

,title
0,Имитационное моделирование систем: искусство и...
1,Введение в имитационное моделирование и язык С...
2,Основы исследования операций
3,Выпуклые структуры и математическая экономика
4,Нелинейное программирование. Теория и алгоритмы
...,...
79,Линейная алгебра и ее применение
80,"Теория синтаксического анализа, перевода и ком..."
81,"Теория синтаксического анализа, перевода и ком..."
82,Введение в программирование на языке Паскаль


###### 2-й способ использует меньший объем памяти и более быстрый

### 5. Вывод количество книг, принадлежащих каждой из категорий.

In [13]:
query_book_category = """
SELECT category.title,
       COUNT(DISTINCT book.id) book_sum
  FROM category
       JOIN
       book ON category.id = book.category_id
 GROUP BY book.category_id
 ORDER BY book_sum;
"""


In [14]:
pd.read_sql(query_book_category, con)

,title,book_sum
0,Экономические науки,198
1,Информационные технологии,201
2,Теория вероятностей и математическая статистика,214
3,Математическая экономия,215


### 6. Вывод количества книг, выпущенных в каждом издательстве.

In [15]:
query_book_publisher = """
SELECT *
  FROM (
           SELECT publisher.name,
                  COUNT(DISTINCT book.id) book_sum
             FROM publisher
                  JOIN
                  book ON publisher.id = book.publisher_id
            GROUP BY book.publisher_id
       )
 WHERE book_sum >= 3
 ORDER BY book_sum DESC;
"""


In [16]:
pd.read_sql(query_book_publisher, con)

,name,book_sum
0,Наука,152
1,Мир,84
2,Финансы и статистика,38
3,ФИЗМАТЛИТ,28
4,Статистика,26
5,Радио и связь,26
6,Питер,23
7,Вильямс,20
8,Высш. шк.,19
9,ЮНИТИ-ДАНА,15


### 7. Вывод списка книг, автором которых является <<С. А. Айвазян>>

In [17]:
query_book_aivasyan = """
SELECT book.title
  FROM book
       JOIN
       authorbook ON authorbook.book_id = book.id AND 
                     authorbook.author_id = (
                                                SELECT id
                                                  FROM author
                                                 WHERE name = 'С. А. Айвазян'
                                            );
"""

In [18]:
pd.read_sql(query_book_aivasyan, con)

,title
0,Социальные индикаторы
1,Прикладная статистика. Основы эконометрики: Уч...
2,Прикладная статистика: Классификации и снижени...
3,Прикладная статистика: Исследование зависимостей
4,Прикладная статистика: Основы моделирования и ...
5,Статистика. Вероятность. Экономика
6,Классификация многомерных наблюдений
7,Прикладная статистика в задачах и упражнениях


### 8. Вывод издательств и количество выпущенных книг за авторством <<С. А. Айвазян>> этими издательствами.

In [19]:
query = """
SELECT name as publisher,
       COUNT(DISTINCT id) as book_sum
  FROM (
           SELECT publisher.name,
                  book.id 
             FROM publisher
                  JOIN
                  book ON publisher.id = book.publisher_id
            GROUP BY book.id
       )
 WHERE id IN (
           SELECT book.id
             FROM book
                  JOIN
                  authorbook ON authorbook.book_id = book.id AND 
                                authorbook.author_id = (
                                                           SELECT id
                                                             FROM author
                                                            WHERE name = 'С. А. Айвазян'
                                                       )
       )
GROUP BY name
ORDER BY book_sum DESC;
"""

In [20]:
pd.read_sql(query, con)

,publisher,book_sum
0,Финансы и статистика,3
1,ЮНИТИ-ДАНА,2
2,Юнити-Дана,1
3,Статистика,1
4,Наука,1
